# nbmodular

> Convert notebooks to modular code.

Convert data science notebooks with poor modularity to fully modular notebooks that are automatically exported as python modules.

## Motivation

In data science, it is usual to develop experimentally and quickly based on notebooks, with little regard to software engineering practices and modularity. It can become challenging to start working on someone else's notebooks with no modularity in terms of separate functions, and a great degree of duplicated code between the different notebooks. This makes it difficult to understand the logic in terms of semantically separate units, see what are the commonalities and differences between the notebooks, and be able to extend, generalize, and configure the current solution.

## Objectives

`nbmodular` is a library conceived with the objective of helping converting the cells of a notebook into separate functions with clear dependencies in terms of inputs and outputs. This is done though a combination of tools which semi-automatically understand the data-flow in the code, based on mild assumptions about its structure. It also helps test the current logic and compare it against a modularized solution, to make sure that the refactored code is equivalent to the original one. 

## Features

- [x] Convert cells to functions.
- [x] The logic of a single function can be written across multiple cells.
- [x] Functions can be either regular functions or unit test functions.
- [x] Functions and tests are exported to separate python modules. 
- [ ] TODO: use nbdev to sync the exported python module with the notebook code, so that changes to the module are reflected back in the notebook.
- [x] Processed cells can continue to operate as cells or be only used as functions.
- [x] A pipeline function is automatically created and updated. This pipeline provides the data-flow from the first to the last function call in the notebook.
- [x] Functions act as nodes in a dependency graph. These nodes can optionally hold the values of local variables for inspection outside of the function. This is similar to having a single global scope, which is the original situation. Since this is memory-consuming, storing local variables is optional.
- [x] Local variables are persisted in disk, so that we may decide to reuse previous results without running the whole notebook. 
- [ ] TODO: Once we are able to construct a graph, we may be able to draw it or show it in text, and pass it to ADG processors that can run functions sequentially or in parallel.
- [ ] TODO: if we have the dependency graph and persisted inputs / outputs, we may decide to only run those cells that are predecessors of the current one, i.e., the ones that provide the inputs needed by the current cell. 
- [ ] TODO: if we associate a hash code to input data, we may only run the cells when the input data changes. Similarly, if we associate a hash code with AST-converted function code, we may only run those cells whose code has been updated. 
- [ ] TODO:  the output of a test cell can be used for assertions, where we require that the current output is the same as the original one.
- [ ] TODO: Compare the result of the pipeline with the result of running the original notebook.
- [ ] TODO: Currently, AST processing is used for assessing whether variables are modified in the cell or are just read. This just gives an estimate. We may want to compare the values of existing variables before and after running the code in the cell. We may also use a type checker such as mypy to assess whether a variable is immutable in the cell (e.g., mark the variable as Final and see if mypy complaints)
- [ ] TODO: have indicated test be used as examples in docstrings. Have optional flag indicate that the next cell's output should be converted to text and included as example output in the docstring.
- [ ] TODO: have the possibility of writing the tests in the same module as the functions, where each test goes after the function that is testing. This can help as a form of documentation for the function, especially if the test code is not included in the function's docstring.

## Install

```sh
pip install nbmodular
```

## Usage

Load ipython extension

In [ ]:
%load_ext nbmodular.core.cell2func

<div style="background-color: rgb(250, 250, 250);">
```python
%load_ext nbmodular.core.cell2func
```
</div>

This allows us to use the following magic commands, among others


- function <name_of_function_to_define>
- print <name_of_previous_function>
- function_info <name_of_previous_function>
- print_pipeline

Let's go one by one

### function

#### Basic usage

The magic command `function` allows to run the code in the cell, as it would be normally done, and at the same time it performs a number of additional steps. Let's go over each one in turn through the following example:

<div style="background-color: rgb(250, 250, 250);">
```python
%%function two_plus_three
a = 2
b = 3
c = a+b
print (f'The result of adding {a}+{b} is {c}')
```

In [ ]:
%%function two_plus_three
#|echo: false
a = 2
b = 3
c = a+b
print (f'The result of adding {a}+{b} is {c}')

The result of adding 2+3 is 5


In [ ]:
(a, b, c)

(2, 3, 5)

As we can see, the previous cell just runs as it would normally do. In addition to this, the code syntax is analyzed using an `ast`, and the result of this analysis is stored in a new object called `two_plus_three_info`. Let's look at some of the information provided by this object.

First, the object stores the list of variables that were created inside this function:

In [ ]:
two_plus_three_info.created_variables

['a', 'b', 'c']

By default, this object also stores the values of those variables:

In [ ]:
two_plus_three_info.current_values

{'a': 2, 'b': 3, 'c': 5}

It stores the names of the variables used by this function and created before calling it:

In [ ]:
two_plus_three_info.previous_variables

[]

In [ ]:
#| hide
assert (a, b, c) == (2, 3, 5)
assert two_plus_three_info.created_variables==['a', 'b', 'c']
assert two_plus_three_info.current_values=={'a': 2, 'b': 3, 'c': 5}
assert two_plus_three_info.previous_variables==[]
assert two_plus_three_info.arguments==[]
assert two_plus_three_info.return_values==[]

In the previous example, there are no previous variables. Let's see a new example with previous variables:

In [ ]:
my_previous_variable=10

<div style="background-color: rgb(250, 250, 250);">
```python
%%function add_100
my_previous_variable = my_previous_variable + 100
print (f'The result of adding 100 to my_previous_variable is {my_previous_variable}')
```
</div>

In [ ]:
%%function add_100
#|echo: false
my_previous_variable = my_previous_variable + 100
print (f'The result of adding 100 to my_previous_variable is {my_previous_variable}')

The result of adding 100 to my_previous_variable is 110


In [ ]:
add_100_info.previous_variables

['my_previous_variable']

`my_previous_variable` is also included in the list of `created_variables`, since a new value for this variable has been generated:

In [ ]:
add_100_info.created_variables

['my_previous_variable']

In [ ]:
add_100_info.arguments

[]

In [ ]:
#| hide
assert my_previous_variable==110
assert add_100_info.created_variables==['my_previous_variable']
assert add_100_info.previous_variables==['my_previous_variable']
assert add_100_info.current_values=={'my_previous_variable': 110}
assert add_100_info.arguments==[]
assert add_100_info.return_values==[]

In addition to the previous analysis, each call to `%%function` creates a new functions, called `two_plus_three` and `add_100` respectively. We can print the code of each function:

<div style="background-color: rgb(250, 250, 250);">
```python
%print two_plus_three
```
</div>

In [ ]:
%print two_plus_three 
#| echo: false

def two_plus_three():
    a = 2
    b = 3
    c = a+b
    print (f'The result of adding {a}+{b} is {c}')



<div style="background-color: rgb(250, 250, 250);">
```python
%print add_100
```
</div>

In [ ]:
%print add_100 
#| echo: false

def add_100(my_previous_variable):
    my_previous_variable = my_previous_variable + 100
    print (f'The result of adding 100 to my_previous_variable is {my_previous_variable}')



The created functions can be called as with any other defined function:

In [ ]:
two_plus_three ()

The result of adding 2+3 is 5


In [ ]:
add_100 (45)

The result of adding 100 to my_previous_variable is 145


All the functions created so far can be printed at once using `print all`: 

<div style="background-color: rgb(250, 250, 250);">
```python
%print all
```
</div>

In [ ]:
%print all
#| echo: false

def two_plus_three():
    a = 2
    b = 3
    c = a+b
    print (f'The result of adding {a}+{b} is {c}')

def add_100(my_previous_variable):
    my_previous_variable = my_previous_variable + 100
    print (f'The result of adding 100 to my_previous_variable is {my_previous_variable}')



And they are also written to a python module with the same name of the notebook (the current notebook being called "index.ipynb"):

In [ ]:
!cat ../nbmodular/index.py

def two_plus_three():
    a = 2
    b = 3
    c = a+b
    print (f'The result of adding {a}+{b} is {c}')

def add_100(my_previous_variable):
    my_previous_variable = my_previous_variable + 100
    print (f'The result of adding 100 to my_previous_variable is {my_previous_variable}')


# -----------------------------------------------------
# pipeline
# -----------------------------------------------------
def index_pipeline (test=False, load=True, save=True, result_file_name="index_pipeline"):
    """Pipeline calling each one of the functions defined in this module."""
    
    # load result
    result_file_name += '.pk'
    path_variables = Path ("index") / result_file_name
    if load and path_variables.exists():
        result = joblib.load (path_variables)
        return result

    two_plus_three ()
    add_100 (my_previous_variable)

    # save result
    result = Bunch ()
    if save:    
        path_variables.parent.mkdir (parents=True, exist_ok=True)
        joblib.dump (res

In [ ]:
#| hide
c = %cell_processor
function_call = ('hybrid', '#|hide\nx = 3\nx = x + 4\nprint (x)\n')
c.process_function_call (*function_call)
c.process_function_call (*function_call)
assert hybrid_info.arguments==[]

7
7


In [ ]:
hybrid_info.previous_variables

['x']

In [ ]:
%print hybrid

def hybrid():
    x = 3
    x = x + 4
    print (x)



In [ ]:
#| hide
c = %cell_processor
function_call = ('hybrid', '#|hide\nx = 3\nx = x + 4\nprint (x)\n')
c.debug_function (call_history=[function_call, function_call])

7
> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(588)process_function_call()
    587     def process_function_call (self, line, cell, add_call=True):
--> 588         call = (line, cell)
    589         if add_call:



ipdb>  b self.create_function_and_run_code


Breakpoint 1 at /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py:721


ipdb>  c


> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(746)create_function_and_run_code()
    745 
--> 746         if restrict_inputs is None:
    747             restrict_inputs = self.restrict_inputs



...

ipdb>  idx


0


ipdb>  this_function.previous_variables


[]


ipdb>  q


#### Dynamic inputs & outputs

So far, none of the created functions return any result. This is because there is no other function that needs any of the variables created inside neither `two_plus_three` nor `add_100`. Let's see what happens when we add a new function that requires the variable `c`, created in `two_plus_three`:

The code in the previous cell runs as it normally would, but and at the same time defines a function named `two_plus_three` which we can show with the magic command `print`:

```ipython
%print two_plus_three 
```

This function is defined in the notebook space, so we can invoke it:

In [ ]:
%print all

def get_initial_values(test=False):
    a = 2
    b = 3
    c = a+b
    print (a+b)



The inputs and outputs of the function change dynamically every time we add a new function cell. For example, if we add a new function `get_d`:

In [ ]:
%%function get_d
d = 10

In [ ]:
%print get_d

def get_d():
    d = 10



And then a function `add_all` that depend on the previous two functions:

In [ ]:
%%function add_all
a = a + d
b = b + d
c = c + d

In [ ]:
f = %function_info add_all

In [ ]:
print(f.code)

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d



In [ ]:
%print add_all

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d



In [ ]:
%print_pipeline --test


from sklearn.utils import Bunch
from pathlib import Path
import joblib
import pandas as pd
import numpy as np

def test_index_pipeline (test=True, prev_result=None, result_file_name="index_pipeline"):
    result = index_pipeline (test=test, load=True, save=True, result_file_name=result_file_name)
    if prev_result is None:
        prev_result = index_pipeline (test=test, load=True, save=True, result_file_name=f"test_{result_file_name}")
    for k in prev_result:
        assert k in result
        if type(prev_result[k]) is pd.DataFrame:    
            pd.testing.assert_frame_equal (result[k], prev_result[k])
        elif type(prev_result[k]) is np.array:
            np.testing.assert_array_equal (result[k], prev_result[k])
        else:
            assert result[k]==prev_result[k]



In [ ]:
%print_pipeline


def index_pipeline (test=False, load=True, save=True, result_file_name="index_pipeline"):

    # load result
    result_file_name += '.pk'
    path_variables = Path ("index") / result_file_name
    if load and path_variables.exists():
        result = joblib.load (path_variables)
        return result

    b, c, a = get_initial_values (test=test)
    d = get_d ()
    add_all (d, b, c, a)

    # save result
    result = Bunch (b=b,c=c,a=a,d=d)
    if save:    
        path_variables.parent.mkdir (parents=True, exist_ok=True)
        joblib.dump (result, path_variables)
    return result



In [ ]:
%print add_all

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d



We can see that the uputs from `two_plus_three` and `get_d` change as needed. We can look at all the functions defined so far by using `print all`:

In [ ]:
%print all

def get_initial_values(test=False):
    a = 2
    b = 3
    c = a+b
    print (a+b)
    return b,c,a

def get_d():
    d = 10
    return d

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d



Similarly the outputs from the last function `add_all` change after we add a other functions that depend on it:

In [ ]:
%%function print_all
print (a, b, c, d)

12 13 15 10


### print

We can see each of the defined functions with `print my_function`, and list all of them with `print all`

In [ ]:
%print all

def get_initial_values(test=False):
    a = 2
    b = 3
    c = a+b
    print (a+b)
    return b,c,a

def get_d():
    d = 10
    return d

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d
    return b,c,a

def print_all(b, d, a, c):
    print (a, b, c, d)



### print_pipeline

As we add functions to the notebook, a pipeline function is defined. We can print this pipeline with the magic `print_pipeline`

In [ ]:
%print_pipeline


def index_pipeline (test=False, load=True, save=True, result_file_name="index_pipeline"):

    # load result
    result_file_name += '.pk'
    path_variables = Path ("index") / result_file_name
    if load and path_variables.exists():
        result = joblib.load (path_variables)
        return result

    b, c, a = get_initial_values (test=test)
    d = get_d ()
    b, c, a = add_all (d, b, c, a)
    print_all (b, d, a, c)

    # save result
    result = Bunch (b=b,d=d,c=c,a=a)
    if save:    
        path_variables.parent.mkdir (parents=True, exist_ok=True)
        joblib.dump (result, path_variables)
    return result



This shows the data flow in terms of inputs and outputs

And run it:

In [ ]:
self = %cell_processor

In [ ]:
self.function_list

[FunctionProcessor with name get_initial_values, and fields: dict_keys(['original_code', 'name', 'call', 'tab_size', 'arguments', 'return_values', 'unknown_input', 'unknown_output', 'test', 'data', 'defined', 'permanent', 'signature', 'norun', 'created_variables', 'loaded_names', 'previous_variables', 'argument_variables', 'read_only_variables', 'posterior_variables', 'all_variables', 'idx', 'previous_values', 'current_values', 'all_values', 'code'])
     Arguments: []
     Output: ['b', 'c', 'a']
     Locals: dict_keys(['a', 'b', 'c']),
 FunctionProcessor with name get_d, and fields: dict_keys(['original_code', 'name', 'call', 'tab_size', 'arguments', 'return_values', 'unknown_input', 'unknown_output', 'test', 'data', 'defined', 'permanent', 'signature', 'norun', 'created_variables', 'loaded_names', 'previous_variables', 'argument_variables', 'read_only_variables', 'posterior_variables', 'all_variables', 'idx', 'previous_values', 'current_values', 'all_values', 'code'])
     Arguments

In [ ]:
%print all

def get_initial_values(test=False):
    a = 2
    b = 3
    c = a+b
    print (a+b)
    return b,c,a

def get_d():
    d = 10
    return d

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d
    return b,c,a

def print_all(b, d, a, c):
    print (a, b, c, d)



In [ ]:
index_pipeline()

{'d': 10, 'b': 13, 'a': 12, 'c': 15}

### function_info

We can get access to many of the details of each of the defined functions by calling `function_info` on a given function name:

In [ ]:
two_plus_three_info = %function_info two_plus_three

This allows us to see:

- The name and value (at the time of running) of the local variables, arguments and results from the function:

In [ ]:
two_plus_three_info.arguments

[]

In [ ]:
two_plus_three_info.current_values

{'a': 2, 'b': 3, 'c': 5}

In [ ]:
two_plus_three_info.return_values

['b', 'c', 'a']

We can also inspect the original code written in the cell...

In [ ]:
print (two_plus_three_info.original_code)

a = 2
b = 3
c = a+b
print (a+b)



the code of the defined function:

In [ ]:
print (two_plus_three_info.code)

def get_initial_values(test=False):
    a = 2
    b = 3
    c = a+b
    print (a+b)
    return b,c,a



.. and the AST trees:

In [ ]:
print (two_plus_three_info.get_ast (code=two_plus_three_info.original_code))

Module(
  body=[
    Assign(
      targets=[
        Name(id='a', ctx=Store())],
      value=Constant(value=2)),
    Assign(
      targets=[
        Name(id='b', ctx=Store())],
      value=Constant(value=3)),
    Assign(
      targets=[
        Name(id='c', ctx=Store())],
      value=BinOp(
        left=Name(id='a', ctx=Load()),
        op=Add(),
        right=Name(id='b', ctx=Load()))),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          BinOp(
            left=Name(id='a', ctx=Load()),
            op=Add(),
            right=Name(id='b', ctx=Load()))],
        keywords=[]))],
  type_ignores=[])
None


In [ ]:
print (two_plus_three_info.get_ast (code=two_plus_three_info.code))

Module(
  body=[
    FunctionDef(
      name='get_initial_values',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(arg='test')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[
          Constant(value=False)]),
      body=[
        Assign(
          targets=[
            Name(id='a', ctx=Store())],
          value=Constant(value=2)),
        Assign(
          targets=[
            Name(id='b', ctx=Store())],
          value=Constant(value=3)),
        Assign(
          targets=[
            Name(id='c', ctx=Store())],
          value=BinOp(
            left=Name(id='a', ctx=Load()),
            op=Add(),
            right=Name(id='b', ctx=Load()))),
        Expr(
          value=Call(
            func=Name(id='print', ctx=Load()),
            args=[
              BinOp(
                left=Name(id='a', ctx=Load()),
                op=Add(),
                right=Name(id='b', ctx=Load()))],
            keywords=[])),
        Return(
       

Now, we can define another function in a cell that uses variables from the previous function.

### cell_processor

This magic allows us to get access to the CellProcessor class managing the logic for running the above magic commands, which can become handy:

In [ ]:
cell_processor = %cell_processor

## Merging function cells

In order to explore intermediate results, it is convenient to split the code in a function among different cells. This can be done by passing the flag `--merge True`

In [ ]:
%%function analyze
x = [1, 2, 3]
y = [100, 200, 300]
z = [u+v for u,v in zip(x,y)]

In [ ]:
z

[101, 202, 303]

In [ ]:
%print analyze

def analyze():
    x = [1, 2, 3]
    y = [100, 200, 300]
    z = [u+v for u,v in zip(x,y)]



In [ ]:
%%function analyze --merge
product = [u*v for u, v in zip(x,y)]

In [ ]:
%print analyze

def analyze():
    x = [1, 2, 3]
    y = [100, 200, 300]
    z = [u+v for u,v in zip(x,y)]
    product = [u*v for u, v in zip(x,y)]



# Test functions

By passing the flag `--test` we can indicate that the logic in the cell is dedicated to test other functions in the notebook. The test function is defined taking the well-known `pytest` library as a test engine in mind. 

This has the following consequences:   
    - The analysis of dependencies is not associated with variables found in other cells.
    - Test functions do not appear in the overall pipeline.
    - The data variables used by the test function can be defined in separate test data cells which in turn are converted to functions. These functions are called at the beginning of the test cell. 
    
Let's see an example

In [ ]:
%%function input_add_all --data --test
a = 5
b = 3
c = 6
d = 7

In [ ]:
add_all(d, a, b, c)

(12, 10, 13)

In [ ]:
%%function add_all --test
# test function add_all
assert add_all(d, a, b, c)==(12, 10, 13)

In [ ]:
%print test_add_all --test

def test_add_all():
    b,c,a,d = test_input_add_all()
    # test function add_all
    assert add_all(d, a, b, c)==(12, 10, 13)



In [ ]:
%print test_input_add_all --test --data

def test_input_add_all(test=False):
    a = 5
    b = 3
    c = 6
    d = 7
    return b,c,a,d



Test functions are written in a separate test module, withprefix `test_`

In [ ]:
!ls ../tests

index.ipynb  test_example.py


# Imports

In order to include libraries in our python module, we can use the magic imports. Those will be written at the beginning of the module:

In [ ]:
%%imports
import pandas as pd

Imports can be indicated separately for the test module by passing the flag `--test`:

In [ ]:
%%imports --test
import matplotlib.pyplot as plt

# Defined functions

Functions can be included already being defined with signature and return values. The only caveat is that, if we want the function to be executed, the variables in the argument list need to be created outside of the function. Otherwise we need to pass the flag --norun to avoid errors:

In [ ]:
%%function --norun
def myfunc (x, y, a=1, b=3):
    print ('hello', a, b)
    c = a+b
    return c

Although the internal code of the function is not executed, it is still parsed using an AST. This allows to provide very tentative *warnings* regarding names not found in the argument list

In [ ]:
%%function --norun
def other_func (x, y):
    print ('hello', a, b)
    c = a+b
    return c

Detected the following previous variables that are not in the argument list: ['b', 'a']


Let's do the same but running the function:

In [ ]:
a=1
b=3

In [ ]:
%%function
def myfunc (x, y, a=1, b=3):
    print ('hello', a, b)
    c = a+b
    return c

hello 1 3


In [ ]:
myfunc (10, 20)

hello 1 3


4

In [ ]:
myfunc_info = %function_info myfunc

In [ ]:
myfunc_info

FunctionProcessor with name myfunc, and fields: dict_keys(['original_code', 'name', 'call', 'tab_size', 'arguments', 'return_values', 'unknown_input', 'unknown_output', 'test', 'data', 'defined', 'permanent', 'signature', 'norun', 'created_variables', 'loaded_names', 'previous_variables', 'argument_variables', 'read_only_variables', 'posterior_variables', 'all_variables', 'idx', 'previous_values', 'current_values', 'all_values', 'code'])
    Arguments: ['x', 'y', 'a', 'b']
    Output: ['c']
    Locals: dict_keys(['c'])

In [ ]:
myfunc_info.c

4

# Storing local variables in memory

By default, when we run a cell function its local variables are stored in a dictionary called `current_values`:

In [ ]:
%%function my_new_function
my_new_local = 3
my_other_new_local = 4

The stored variables can be accessed by calling the magic `function_info`:

In [ ]:
my_new_function_info = %function_info my_new_function

In [ ]:
my_new_function_info.current_values

{'my_new_local': 3, 'my_other_new_local': 4}

This default behaviour can be overriden by passing the flag `--not-store`

In [ ]:
%%function my_second_new_function --not-store
my_second_variable = 100
my_second_other_variable = 200

In [ ]:
my_second_new_function_info = %function_info my_second_new_function

In [ ]:
my_second_new_function_info.current_values

{}

# (Un)packing Bunch I/O

In [ ]:
%load_ext nbmodular.core.cell2func

In [ ]:
from sklearn.utils import Bunch

In [ ]:
%%function bunch_data
x = Bunch (a=1, b=2)

In [ ]:
%%function bunch_processor --unpack-bunch x --include-input "day=1"
c = 3
a = 4

In [ ]:
%print bunch_processor

def bunch_processor(x, day):
    a = x["a"]
    b = x["b"]
    c = 3
    a = 4
    x["a"] = a
    x["c"] = c
    x["day"] = day
    return x



# Function's info object holding local variables

In [ ]:
#| hide
import pandas as pd

In [ ]:
df = pd.DataFrame (dict(Year=[1,2,3], Month=[1,2,3], Day=[1,2,3]))
fy = '2023'

In [ ]:
%%function
def days (df, fy, x=1, /, y=3, *, n=4):
    df_group = df.groupby(['Year','Month']).agg({'Day': lambda x: len (x)})
    df_group = df.reset_index()
    print ('other args: fy', fy, 'x', x, 'y', y)
    return df_group

other args: fy 2023 x 1 y 3
Stored the following local variables in the days current_values dictionary: ['df_group']
Detected the following previous variables that are not in the argument list: ['x', 'df', 'fy']


An info object with name <function_name>_info is created in memory, and can be used to get access to local variables

In [ ]:
days_info.df_group

,index,Year,Month,Day
0,0,1,1,1
1,1,2,2,2
2,2,3,3,3


There is more information in this object: previous variables, code, etc.

In [ ]:
days_info.current_values

{'df_group':    index  Year  Month  Day
 0      0     1      1    1
 1      1     2      2    2
 2      2     3      3    3}

In [ ]:
days_info

FunctionProcessor with name days, and fields: dict_keys(['original_code', 'name', 'call', 'tab_size', 'arguments', 'return_values', 'unknown_input', 'unknown_output', 'test', 'data', 'defined', 'permanent', 'signature', 'not_run', 'previous_values', 'current_values', 'returns_dict', 'returns_bunch', 'unpack_bunch', 'include_input', 'exclude_input', 'include_output', 'exclude_output', 'store_locals_in_disk', 'created_variables', 'loaded_names', 'previous_variables', 'argument_variables', 'read_only_variables', 'posterior_variables', 'all_variables', 'idx'])
    Arguments: ['df', 'fy', 'x', 'y']
    Output: ['df_group']
    Locals: dict_keys(['df_group'])

The function can also be called directly:

In [ ]:
days (df*100, 100, x=4)

other args: fy 100 x 4 y 3


,index,Year,Month,Day
0,0,100,100,100
1,1,200,200,200
2,2,300,300,300
